In [ ]:
import requests

def get_all_binance_symbols():
    """
    Fetches all available trading symbols from Binance API and returns them in a structured format.
    Returns a list of dictionaries with keys: Symbol, Name, Exchange
    """
    try:
        # Binance API endpoint for exchange information
        url = "https://api.binance.com/api/v3/exchangeInfo"
        
        # Make the GET request
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse the JSON response
        data = response.json()
        
        # Create structured data
        symbols_data = [
            {
                "Symbol": symbol['symbol'],
                "Name": symbol['symbol'],
                "Asset Class": "Binance"
            }
            for symbol in data['symbols']
            if symbol['status'] == 'TRADING'  # Only include active symbols
        ]
        
        return symbols_data
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Binance API: {e}")
        return []
    

In [ ]:
symbols = get_all_binance_symbols()
print(f"Total symbols: {len(symbols)}")
# print("First 10 symbols:", symbols[:10])
    
    # Optional: Save to a file
    # with open('binance_symbols.txt', 'w') as f:
    #     for symbol in symbols:
    #         f.write(f"{symbol}\n")
    # print("Symbols saved to binance_symbols.txt")

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_binance_ohlc(symbol, start_date, end_date, interval='1d'):
    """
    Fetch OHLC data from Binance API for a given symbol and date range
    
    Parameters:
        symbol (str): Trading pair symbol (e.g., 'BTCUSDT')
        start_date (str/datetime): Start date in format 'YYYY-MM-DD' or datetime object
        end_date (str/datetime): End date in format 'YYYY-MM-DD' or datetime object
        interval (str): Kline interval (default '1d' - 1 day)
                       Options: '1m', '5m', '15m', '30m', '1h', '2h', '4h', 
                                '6h', '8h', '12h', '1d', '3d', '1w', '1M'
    
    Returns:
        pd.DataFrame: DataFrame with OHLC data and volume
    """
    # Convert dates to timestamp in milliseconds
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    start_ts = int(start_date.timestamp() * 1000)
    end_ts = int(end_date.timestamp() * 1000)
    
    base_url = "https://api.binance.com/api/v3/klines"
    all_data = []
    
    current_start = start_ts
    max_records_per_request = 1000  # Binance's limit
    
    try:
        while current_start < end_ts:
            # Calculate end timestamp for this batch
            current_end = min(current_start + max_records_per_request * get_interval_ms(interval), end_ts)
            
            params = {
                'symbol': symbol,
                'interval': interval,
                'startTime': current_start,
                'endTime': current_end,
                'limit': max_records_per_request
            }
            
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if not data:
                break
                
            all_data.extend(data)
            
            # Move window forward
            current_start = int(data[-1][0]) + get_interval_ms(interval)
            
        # Process the data into a DataFrame
        columns = [
            'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
            'Close time', 'Quote asset volume', 'Number of trades',
            'Taker buy base volume', 'Taker buy quote volume', 'Ignore'
        ]
        
        df = pd.DataFrame(all_data, columns=columns)
        
        # Convert timestamp to datetime
        df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
        df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
        
        # Convert strings to numeric values
        numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, axis=1)
        
        # Filter only the relevant columns
        df = df[['Open time', 'Open', 'High', 'Low', 'Close', 'Volume']]
        
        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching OHLC data: {e}")
        return pd.DataFrame()

def get_interval_ms(interval):
    """Convert interval string to milliseconds"""
    unit = interval[-1]
    value = int(interval[:-1])
    
    if unit == 'm':
        return value * 60 * 1000
    elif unit == 'h':
        return value * 60 * 60 * 1000
    elif unit == 'd':
        return value * 24 * 60 * 60 * 1000
    elif unit == 'w':
        return value * 7 * 24 * 60 * 60 * 1000
    elif unit == 'M':
        return value * 30 * 24 * 60 * 60 * 1000  # Approximate
    else:
        return 0

    

In [ ]:
# Fetch BTC/USDT daily data from Jan 1 to Dec 31, 2023
ohlc_data = fetch_binance_ohlc(
    symbol='BTCUSDT',
    start_date='2025-05-30',
    end_date='2025-06-30',
    interval='1h'
)

print(f"Fetched {len(ohlc_data)} records")
print(ohlc_data.head())

In [ ]:
def getPivot(data,index,interval=2):
    resp = {
        "message": None,
        "index":data.at[index,"Open time"],
        "startIndex": data.at[index - interval,"Open time"],
        "endIndex": data.at[index + interval,"Open time"],
        "interval":interval,
        "isSwingHigh": None,
        "isSwingLow": None,
        "valid" : False,
        # "before": [],
        # "after":[]
    }
    df=data.copy()
    start = index - interval 
    end = index + interval 
    sliceData = []
    current={}
    before=[]
    after=[]

    if start >= 0 and end < len(df):
        sliceData = df.loc[start:end, ['Open time',"High", "Low"]]  # end+1 because slicing is exclusive
    else:
        resp["message"] = f"Cannot slice - would go out of bounds (start: {start}, end: {end}, df length: {len(df)})"

    if resp["message"] == None:
        # current = sliceData.iloc[index]
        current = sliceData[sliceData.index==index]
        # All Highs before current index
        before = sliceData.loc[:index - 1 ]
        # resp["before"] = sliceData.loc[:index - 1 ]
        # All Highs after current index
        after = sliceData.loc[index + 1:]
        # resp["after"] = sliceData.loc[index + 1:]

        resp["isSwingHigh"] =  current['High'].item() if (current['High'].item()>before['High'].max().item()) and (current['High'].item()>after["High"].max().item()) else None
        resp["isSwingLow"] = current['Low'].item() if (current['Low'].item()<before['Low'].min().item()) and (current['Low'].item()<after["Low"].min().item()) else None
        
        if resp["isSwingHigh"] or resp["isSwingLow"]:
            resp["valid"] = True
    # print(sliceData)
    return resp

def getPivots(data,interval=2,beginIndex=None,stopIndex=None):

    resp={
        "message":None,
        "data":[],
        "start_idx" : data.at[interval,"Open time"],
        "end_idx" : data.at[len(data) - interval - 1,"Open time"],
        "interval":interval,
        "beginIndex": beginIndex,
        "stopIndex":stopIndex

    }
    
    start_idx = interval
    end_idx = len(data) - interval 
    minLen = (interval*2)+1

    if (minLen>len(data)):
        resp["message"]=f"Data only has length of {len(data)} which is min length is {minLen}"
        return resp
        

    if beginIndex :
        if beginIndex>start_idx and (( end_idx-beginIndex)>=minLen):
            start_idx=beginIndex
        
    
    if stopIndex and (( stopIndex-start_idx)>=minLen):
        if stopIndex<end_idx:
            end_idx=stopIndex

    for idx in range(start_idx, end_idx ):
        result = getPivot(data, idx, interval)
        resp["data"].append(result)
    
    return resp

In [ ]:
x=getPivots(ohlc_data,3)
# print(x['data'],sep=',')


In [ ]:
df=pd.DataFrame(x['data'])

In [ ]:
ohlc_data["swingHigh"]=[None ]* len(ohlc_data)
swingHigh=df[df["isSwingHigh"].notna()]
# ohlc_data[ohlc_data["Open time"]==swingHigh["index"]]

In [ ]:
# ohlc_data["Open time"].values
# swingHigh["index"].values
import numpy as np
# Find matching indices
mask = np.isin(ohlc_data["Open time"], swingHigh["index"])

# Make sure the lengths match
if sum(mask) == len(swingHigh["isSwingHigh"]):
    ohlc_data.loc[mask, "swingHigh"] = swingHigh["isSwingHigh"].values
else:
    print(f"Warning: {sum(mask)} matches found but {len(swingHigh['isSwingHigh'])} values to assign")

# matches = set(ohlc_data["Open time"].values) & set(swingHigh["index"].values)

In [ ]:
def getSwingHighBreak(swings,data):
    # resp =swings.copy()
    # resp['breakHigh'] = [None]*len(resp)
    resp=list()
    for idx in range(0, len(swings) ):
        testHigh=swingHigh.iloc[idx][["isSwingHigh","index"]]
        res= data[(data["Close"] > testHigh["isSwingHigh"]) & (data['Open time'] > testHigh['index'])]['Open time']
        if len(res)>0 :
            resp.append({"p1":{"time":testHigh["index"],"price":testHigh["isSwingHigh"]},"p2":{"time":res.iloc[0],"price":testHigh["isSwingHigh"]}})
    return resp

In [21]:
getSwingHighBreak(swingHigh,ohlc_data)

[{'p1': {'time': Timestamp('2025-05-30 05:00:00'),
   'price': np.float64(106313.12)},
  'p2': {'time': Timestamp('2025-06-03 01:00:00'),
   'price': np.float64(106313.12)}},
 {'p1': {'time': Timestamp('2025-05-30 12:00:00'),
   'price': np.float64(106000.0)},
  'p2': {'time': Timestamp('2025-06-03 00:00:00'),
   'price': np.float64(106000.0)}},
 {'p1': {'time': Timestamp('2025-05-31 05:00:00'),
   'price': np.float64(103911.1)},
  'p2': {'time': Timestamp('2025-05-31 12:00:00'),
   'price': np.float64(103911.1)}},
 {'p1': {'time': Timestamp('2025-05-31 15:00:00'),
   'price': np.float64(104846.15)},
  'p2': {'time': Timestamp('2025-05-31 21:00:00'),
   'price': np.float64(104846.15)}},
 {'p1': {'time': Timestamp('2025-05-31 22:00:00'),
   'price': np.float64(104900.0)},
  'p2': {'time': Timestamp('2025-06-01 16:00:00'),
   'price': np.float64(104900.0)}},
 {'p1': {'time': Timestamp('2025-06-01 04:00:00'),
   'price': np.float64(104729.99)},
  'p2': {'time': Timestamp('2025-06-01 16:00

In [ ]:
def getGap(data,index):
    resp = {
        "message": None,
        "index":data.at[index,"Open time"],
        'isBuy':False,
        'isGap':False,
        "Pre": None,
        "Post": None,
        "valid" : False,
        # "before": [],
        # "after":[]
    }
    df=data.copy()
    current={}
    before=[]
    after=[]

    # if start >= 0 and end < len(df):
    #     sliceData = df.loc[start:end, ['Open time',"High", "Low",'Close','Open']]  # end+1 because slicing is exclusive
    # else:
    #     resp["message"] = f"Cannot slice - would go out of bounds (start: {start}, end: {end}, df length: {len(df)})"

    # if resp["message"] == None:
    # current = sliceData.iloc[index]
    current = df.iloc[index]
    # All Highs before current index
    before = df.iloc[index - 1 ]
    # resp["before"] = sliceData.loc[:index - 1 ]
    # All Highs after current index
    after = df.iloc[index + 1]
    # resp["after"] = sliceData.loc[index + 1:]
    if(current["Open"]<current['Close']):
        resp['isBuy']=True
        resp['isGap'] = before['High']<after['Low']
        if resp["isGap"]:
            resp["Pre"] = before['High']
            resp["Post"] =after['Low']
    else:
        resp['isGap'] = before['Low']>after['High']
        if resp["isGap"]:
            resp["Pre"] = before['High']
            resp["Post"] =after['Low']
    
    if resp["isGap"]:
        resp["valid"] = True
    return resp
    # print(sliceData)
    
def getGapEnd(data,result):
    resp = {
        "message": None,
        "index":result["index"],
        'isBuy':result['isBuy'],
        "Pre": result['Pre'],
        "Post": result['Post'],
        "end" : None,
        # "before": [],
        # "after":[]
    }

    print(data.head())

    if result["isBuy"] :
        res= data[(data["Close"] < result["Post"]) & (data['Open time'] > result["index"])]['Open time']
    else:
        res= data[(data["Close"] > result["Post"]) & (data['Open time'] > result["index"])]['Open time']
    if len(res)>0 :
        resp['end']=res.iloc[0]
    else:
        resp['end']=data.iloc[-1]['Open time']
    
    return resp

def getGaps(data):

    resp={
        "message":None,
        "data":[],
    }
    
    start_idx = 1
    end_idx = len(data) - 1 
    minLen = 3

    if (minLen>len(data)):
        resp["message"]=f"Data only has length of {len(data)} which is min length is {minLen}"
        return resp
        

    for idx in range(start_idx, end_idx ):
        result = getGap(data, idx)
        if result['isGap']:
            resp["data"].append(getGapEnd(data,result))
    
    return resp

In [39]:
gp=getGaps(ohlc_data)
dfGP=pd.DataFrame(gp['data'])
test=dfGP[dfGP["isGap"]]

            Open time       Open       High        Low      Close      Volume  \
0 2025-05-29 22:00:00  106443.02  106460.00  105906.65  105929.07   391.58161   
1 2025-05-29 23:00:00  105929.08  106078.47  105322.86  105589.75   853.19262   
2 2025-05-30 00:00:00  105589.75  106214.93  104600.50  104939.99  1835.85842   
3 2025-05-30 01:00:00  104940.00  105925.92  104940.00  105666.66  1178.59954   
4 2025-05-30 02:00:00  105666.67  106172.30  105650.00  106000.00   767.82001   

  swingHigh  
0      None  
1      None  
2      None  
3      None  
4      None  


KeyError: 'time'